In [31]:
import subprocess
import os
import time
import csv

if os.name == 'nt':  # Windows
    execution_file = os.path.join("bin", "mmul_bench.exe")
else:  # Linux/Macc
    execution_file = os.path.join(".", "bin", "mmul_bench")

algos = ['ijk', 'jik', 'kij', 'ikj', 'jki', 'kji']

def run_alg(N, alg):
    cmd = [execution_file, str(N), alg]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return result.stdout, result.stderr

def format_output(result):
    stdout_lines = result[0].strip().split('\n')
    stderr_lines = result[1].strip().split('\n')
    print("Résultats stdout :")
    for line in stdout_lines:
        print(line)
    print("\nRésultats stderr :")
    for line in stderr_lines:
        print(line)

def generate_range(min=16, max=1024, multiplier=2, step=100, use_step=False):
    range = []

    if use_step:
        n = min
        while n <= max:
            range.append(n)
            n += step
    else:
        n = min
        while n <= max:
            range.append(n)
            n *= multiplier

    return range

def main():
    print("Exécution du programme de benchmark pour la multiplication de matrices...\n")
    print(f"Fichier d'exécution : {execution_file}\n")
    print("Algorithmes disponibles :", ", ".join(algos))
    print("\n" + "=" * 40)

    range_N = generate_range(min=100, max=1000, use_step=True)

    # Ouvre le fichier CSV en écriture
    with open("resultats.csv", "w", newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["N", "algo", "time", "iteration"])  # En-tête

        #for alg in algos:
        for alg in algos:
            print(f"\nAlgorithme : {alg}")
            for N in range_N:
                print(f"\nExécution pour N = {N} :")
                result = run_alg(N, alg)
                stdout_lines = result[0].strip().split('\n')[0]
                print(stdout_lines)

                algo = stdout_lines.split(':')[0].strip()
                time_taken = stdout_lines.split(':')[1].strip().split("μs")[0]
                iteration = stdout_lines.split(':')[2].strip()
                writer.writerow([N, algo, time_taken, iteration])
            
                format_output(result)
                print("-" * 40)
            print("\n" + "=" * 40)
            print("\n")

main()


Exécution du programme de benchmark pour la multiplication de matrices...

Fichier d'exécution : ./bin/mmul_bench

Algorithmes disponibles : ijk, jik, kij, ikj, jki, kji


Algorithme : ijk

Exécution pour N = 100 :
ijk:1172273.750000μs:1074
Résultats stdout :
ijk:1172273.750000μs:1074

Résultats stderr :
enabling: 'ijk'
31945.423350
----------------------------------------

Exécution pour N = 200 :
ijk:1389782.000000μs:135
Résultats stdout :
ijk:1389782.000000μs:135

Résultats stderr :
enabling: 'ijk'
6732.840205
----------------------------------------

Exécution pour N = 300 :
ijk:1516801.500000μs:40
Résultats stdout :
ijk:1516801.500000μs:40

Résultats stderr :
enabling: 'ijk'
2921.399837
----------------------------------------

Exécution pour N = 400 :
ijk:1517448.750000μs:17
Résultats stdout :
ijk:1517448.750000μs:17

Résultats stderr :
enabling: 'ijk'
1656.306190
----------------------------------------

Exécution pour N = 500 :
ijk:1460333.500000μs:9
Résultats stdout :
ijk:1460

In [35]:
import pandas as pd
from plotnine import ggplot, aes, geom_line, labs, theme_minimal, scale_x_continuous, scale_y_continuous

# Paramètres machine (à adapter)
FREQ_CPU_HZ = 2_500_000_000  # 2.5 GHz
NB_COEURS = 4
OP_PAR_CYCLE = 1

# Charger les données du CSV
df = pd.read_csv("resultats.csv")
df["N"] = pd.to_numeric(df["N"])
df["time"] = pd.to_numeric(df["time"])
df["iteration"] = pd.to_numeric(df["iteration"])

df["nb_ops"] = 2 * (df["N"] ** 3) * df["iteration"]
df["FLOPS"] = df["nb_ops"] / (df["time"] / 1_000_000)

FLOPS_MAX = FREQ_CPU_HZ * NB_COEURS * OP_PAR_CYCLE

plot = (
    ggplot(df, aes(x="N", y="FLOPS", color="algo"))
    + geom_line()
    + scale_x_continuous(trans='log2')
    + scale_y_continuous(trans='log10')
    + labs(
        title=f"Performance réelle (FLOPS) selon N (log2) — Axe Y log10\nMax théorique : {FLOPS_MAX:.2e}",
        x="Taille N (log2)",
        y="FLOPS (log10)"
    )
    + theme_minimal()
)
plot.save("resultats_flops_plot.png", width=10, height=6, dpi=300)

/home/natha/benchmarking/bench/.venv/lib/python3.13/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 10 x 6 in image.
/home/natha/benchmarking/bench/.venv/lib/python3.13/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: resultats_flops_plot.png
